In [1]:
import numpy as np
import cv2
import scipy
import matplotlib.pylab as plt
from PIL import Image
from PIL import ImageFilter

In [2]:
img = cv2.imread('./images/final_exam/Templates/Template-1.png')
def preprocessing(img) :
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    return (img, img_gray) 


In [3]:
img =cv2.bilateralFilter(img, 15,75,75) 
template_img, template_gray = preprocessing(img) 

In [4]:
sift = cv2.SIFT_create() 
bf = cv2.BFMatcher()

In [5]:
vid = cv2.VideoCapture('./videos/final_exam/Dataset-2/right_output.avi')
def feature_object_detection(template_img, template_gray, frame,frame_gray, min_match_number) :
    template_kpts, template_desc = sift.detectAndCompute(template_gray, None) # detect หาจุดระหว่าง template_kpts กับ template_desc ในภาพ template_gray
    query_kpts, query_desc = sift.detectAndCompute(frame_gray, None) # detect หาจุดระหว่าง query_kpts กับ query_desc ใน frame_gray
    matches = bf.knnMatch(template_desc, query_desc, k=2)  # ทำการ match กันระหว่าง template_desc กับ query_desc
    good_matches = list() 
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.8*n.distance : 
            good_matches.append(m)
            good_matches_list.append([m])
    
    if len(good_matches) > min_match_number :
       
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)
        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5) #5
        h, w = template_img.shape[:2] 
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2) 
        transformed_box = cv2.perspectiveTransform(template_box, H) 
        detected_img = cv2.polylines(frame, [np.int32(transformed_box)], True, (0,0,255), 3, cv2.LINE_AA) 
        drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)
        
        cv2.imshow('detected_img', detected_img)
        cv2.imshow('drawmatch_img',drawmatch_img)
        return detected_img, drawmatch_img 
    else :
        return 

In [6]:
while vid.isOpened() :
    ret, frame = vid.read() 
    ret_1, frame_1 = vid.read()
    freme_1=cv2 .morphologyEx ( frame_1 ,  cv2 . MORPH_OPEN  , (5,5))
    frame_1 = cv2.bilateralFilter(frame_1, 3,75, 75)
    freme_1=cv2 .morphologyEx ( frame_1 ,  cv2.MORPH_GRADIENT  , (5,5))

    if ret :
        
        frame_1_gray = cv2.cvtColor( frame_1, cv2.COLOR_BGR2GRAY)
        frame_1_gray = frame_1_gray+frame_1_gray
        detected, drawmatch =  feature_object_detection(template_img, template_gray,frame,frame_1_gray,8) 
        if cv2.waitKey(int(1000/24)) & 0xFF == ord('q') : 
            break
    else :
        break
cv2.destroyAllWindows()